<a href="https://colab.research.google.com/github/cabbi120101/final_multi/blob/main/Try/%EB%A1%A4_%EC%B1%94%ED%94%BC%EC%96%B8%EC%9C%BC%EB%A1%9C_%EC%A1%B0%ED%95%A9%EC%83%81_%EC%8A%B9%EB%A5%A0%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mp
from pandas.io.json import json_normalize

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
fm = mp.font_manager.FontManager()
plt.rcParams['font.family'] = 'Malgun Gothic'

In [84]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/동욱코드모음/마지막프로젝트/3주차/시도/champ_addSk.csv', encoding='utf-8', engine='python', thousands=',')
df.head()

,Unnamed: 0,version,id,key,name,title,blurb,info,image,tags,partype,stats,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,skins,lore,allytips,enemytips,spells,passive,recommended,skillQ,skillW,skillE,skillR
0,0,11.17.1,Aatrox,266,Aatrox,the Darkin Blade,"Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually bec...","{'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4}","{'full': 'Aatrox.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 0, 'y': 0, 'w': 48, ...","['Fighter', 'Tank']",Blood Well,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperlevel': 0, 'movespeed': 345, 'armor': 38, 'armorper...",8,4,3,4,580.0,90,0.0,0.0,345,38,3.25,32.0,1.25,175,3.00,1.00,0.000,0.0,0,0,60.0,5.00,2.500,0.651,"[{'id': '266000', 'num': 0, 'name': 'default', 'chromas': False}, {'id': '266001', 'num': 1, 'na...","Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually bec...",['Use Umbral Dash while casting The Darkin Blade to increase your chances of hitting the enemy.'...,"[""Aatrox's attacks are very telegraphed, so use the time to dodge the hit zones."", ""Aatrox's Inf...","[{'id': 'AatroxQ', 'name': 'The Darkin Blade', 'description': 'Aatrox slams his greatsword down,...","{'name': 'Deathbringer Stance', 'description': ""Periodically, Aatrox's next basic attack deals b...",[],"{'id': 'AatroxQ', 'name': 'The Darkin Blade', 'description': 'Aatrox slams his greatsword down, ...","{'id': 'AatroxW', 'name': 'Infernal Chains', 'description': 'Aatrox smashes the ground, dealing ...","{'id': 'AatroxE', 'name': 'Umbral Dash', 'description': 'Passively, Aatrox heals when damaging e...","{'id': 'AatroxR', 'name': 'World Ender', 'description': 'Aatrox unleashes his demonic form, fear..."
1,1,11.17.1,Ahri,103,Ahri,the Nine-Tailed Fox,"Innately connected to the latent power of Runeterra, Ahri is a vastaya who can reshape magic int...","{'attack': 3, 'defense': 4, 'magic': 8, 'difficulty': 5}","{'full': 'Ahri.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 48, 'y': 0, 'w': 48, '...","['Mage', 'Assassin']",Mana,"{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mpperlevel': 25, 'movespeed': 330, 'armor': 21, 'armor...",3,4,8,5,526.0,92,418.0,25.0,330,21,3.50,30.0,0.50,550,5.50,0.60,8.000,0.8,0,0,53.0,3.00,2.000,0.668,"[{'id': '103000', 'num': 0, 'name': 'default', 'chromas': False}, {'id': '103001', 'num': 1, 'na...","Innately connected to the latent power of Runeterra, Ahri is a vastaya who can reshape magic int...","['Use Charm to set up your combos, it will make landing Orb of Deception and Fox-Fire dramatical...","[""Ahri's survivability is dramatically reduced when her Ultimate, Spirit Rush, is down."", ""Stay ...","[{'id': 'AhriOrbofDeception', 'name': 'Orb of Deception', 'description': 'Ahri sends out and pul...","{'name': 'Essence Theft', 'description': 'When Ahri strikes 9 enemies with her abilities, her ne...",[],"{'id': 'AhriOrbofDeception', 'name': 'Orb of Deception', 'description': 'Ahri sends out and pull...","{'id': 'AhriFoxFire', 'name': 'Fox-Fire', 'description': 'Ahri gains a brief burst of movement s...","{'id': 'AhriSeduce', 'name': 'Charm', 'description': 'Ahri blows a kiss that damages and charms ...","{'id': 'AhriTumble', 'name': 'Spirit Rush', 'description': 'Ahri dashes forward and fires essenc..."
2,2,11.17.1,Akali,84,Akali,the Rogue Assassin,"Abandoning the Kinkou Order and her title of the Fist of Shadow, Akali now strikes alone, ready ...","{'attack': 5, 'defense': 3, 'magic': 8, 'difficulty': 7}","{'full': 'Akali.png', 'sprite': 'champion0.png', 'group': 'champion', 'x': 96, 'y': 0, 'w': 48, ...",['Assassin'],Energy,"{'hp': 500, 'hpperlevel': 105, 'mp': 200, 'mpperlevel': 0, 'movespeed': 345, 'armor': 23, 'armor...",5,3,8,7,500.0,

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 47 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            156 non-null    int64  
 1   version               156 non-null    object 
 2   id                    156 non-null    object 
 3   key                   156 non-null    int64  
 4   name                  156 non-null    object 
 5   title                 156 non-null    object 
 6   blurb                 156 non-null    object 
 7   info                  156 non-null    object 
 8   image                 156 non-null    object 
 9   tags                  156 non-null    object 
 10  partype               156 non-null    object 
 11  stats                 156 non-null    object 
 12  attack                156 non-null    int64  
 13  defense               156 non-null    int64  
 14  magic                 156 non-null    int64  
 15  difficulty            1

## 컬럼설명

1. version : 버전
2. id : 챔피언 이름 
3. key : 키(무슨키인지 모르겠음)  
4. name : 챔피언 이름
5. title : 별명
6. blurb : 선전
7. info : 챔피언 정보
8. image : 챔피언들 어디에있는지 위치, 이미지 
9. tags : 챔피언 역할군
10. partype : 챔피언의 자원 
11. stats : 통계
12. attack : 공격
13. defense : 방어  
14. magic : 마력  
15. difficulty : 난이도 
16. hp : 체력량
17. hpperlevel : 레벨에 따른 체력 증가량   
18. mp : 마나량 (마나:스킬을 쓸수있는 총량)
19. mpperlevel : 레벨에 따른 마나 증가량
20. movespeed : 이동속도  
21. armor : 갑옷  
22. armorperlevel : 갑옷레벨
23. spellblock : 주문차단
24. spellblockperlevel : 주문차단레벨
25. attackrange : 공격범위  
26. hpregen : 체력회복량
27. hpregenperlevel : 레벨에 따른 체력회복량
28. mpregen : 마나회복량
29. mpregenperlevel : 레벨에 따른 마나회복량
30. crit : 치명타
31. critperlevel : 레벨에 따른 치명타   
32. attackdamage : 공격 피해          
33. attackdamageperlevel : 공격 데미지당 레벨
34. attackspeedperlevel : 공격 속도당 레벨
35. attackspeed : 공격 속도 오프셋         
36. skins : 스킨(챔피언별 단계)        
37. lore : 배경                 
38. allytips : 동맹국             
39. enemytips : 귓속말            
40. spells : 주문                 
41. passive : 패시브              
42. recommended : 추천          
43. skillQ : 스킬Q               
44. skillW : 스킬W                
45. skillE : 스킬E                
46. skillR : 스킬R                

In [86]:
len(df['tags'].iloc[0])

19

In [87]:
pd.set_option('max_colwidth', 100)
df = df[['id', 'tags']]

## 텍스트 정보들을 피처 벡터화

In [88]:
df['tags']

0           ['Fighter', 'Tank']
1          ['Mage', 'Assassin']
2                  ['Assassin']
3      ['Marksman', 'Assassin']
4           ['Tank', 'Support']
                 ...           
151                ['Assassin']
152                    ['Mage']
153         ['Support', 'Mage']
154         ['Mage', 'Support']
155         ['Mage', 'Support']
Name: tags, Length: 156, dtype: object

In [90]:
# genres 문자열 데이터를 리스트 객체로 변환
from ast import literal_eval

df['tags'] = df['tags'].apply(literal_eval)

In [94]:
# 리스트인 장르 칼럼을 문자열로 변환

from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 단어 단위가 구분되는 문자열로 변환
# join() 메소드 사용해서 리스트의 각 원소들을 공백으로 결합해서 문자열로 변환
df['new_tags'] = df['tags'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
tags_mat = count_vect.fit_transform(df['new_tags'])
print(tags_mat.shape)

(156, 27)


In [95]:
tags_mat.toarray()[1]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [96]:
df['new_tags']

0           Fighter Tank
1          Mage Assassin
2               Assassin
3      Marksman Assassin
4           Tank Support
             ...        
151             Assassin
152                 Mage
153         Support Mage
154         Mage Support
155         Mage Support
Name: new_tags, Length: 156, dtype: object

## 코사인 유사도로 챔피언별 유사도 계산

### 태그에 따른 챔피언별 코사인 유사도 추출

In [109]:
# 사이킷런의 cosine_similarity() 함수 이용해서 코사인 유사도 계산

from sklearn.metrics.pairwise import cosine_similarity

tags_sim = cosine_similarity(tags_mat, tags_mat)
print(tags_sim.shape)  # 유사도 행렬 크기
print(tags_sim[:5])    # 앞 5개만 출력

(156, 156)
[[1.         0.         0.         0.         0.33333333 0.33333333
  0.         0.         0.         0.         0.         0.
  0.         0.66666667 0.         0.33333333 0.         1.
  0.         0.33333333 0.         1.         0.33333333 0.
  1.         0.33333333 0.33333333 0.         0.         0.
  0.33333333 0.33333333 0.33333333 0.57735027 1.         1.
  0.33333333 0.         0.33333333 1.         0.         1.
  0.33333333 0.         0.         0.66666667 0.33333333 0.33333333
  0.         0.         0.         0.         0.         0.
  0.         0.         0.33333333 0.33333333 0.         0.
  0.         1.         0.         0.         0.33333333 0.33333333
  0.33333333 0.         0.         0.         0.         0.66666667
  0.         0.33333333 0.33333333 0.         1.         0.57735027
  0.         0.         1.         0.66666667 0.         0.
  0.33333333 0.66666667 1.         0.         0.66666667 0.33333333
  0.66666667 0.         0.33333333 0.    

In [110]:
type(tags_sim)

numpy.ndarray

In [111]:
tags_sim.argsort()

array([[155,  78, 154, ..., 110, 142,   0],
       [  0,  96,  95, ...,   1,  72, 118],
       [  0, 106, 105, ..., 151,   2, 122],
       ...,
       [  0, 103, 100, ..., 116,  69, 153],
       [  0, 103, 100, ...,  52, 121, 155],
       [  0, 103, 100, ...,  52, 121, 155]])

In [112]:
# 행으로 내림차순 정렬
tags_sim.argsort()[::-1]

array([[  0, 103, 100, ...,  52, 121, 155],
       [  0, 103, 100, ...,  52, 121, 155],
       [  0, 103, 100, ..., 116,  69, 153],
       ...,
       [  0, 106, 105, ..., 151,   2, 122],
       [  0,  96,  95, ...,   1,  72, 118],
       [155,  78, 154, ..., 110, 142,   0]])

In [113]:
# 행은 그대로, 열로 내림차순 정렬
tags_sim.argsort()[:, ::-1]

array([[  0, 142, 110, ..., 154,  78, 155],
       [118,  72,   1, ...,  95,  96,   0],
       [122,   2, 151, ..., 105, 106,   0],
       ...,
       [153,  69, 116, ..., 100, 103,   0],
       [155, 121,  52, ..., 100, 103,   0],
       [155, 121,  52, ..., 100, 103,   0]])

In [114]:
tags_sim.argsort()[:, ::-1][1]

array([118,  72,   1,  63,  83,  55,  27,  54,  67,   7, 129, 108,  18,
       136,  10, 122,  14,  59, 135,   2, 152, 139, 140,  53, 144, 151,
        29,  79,  92,  32,  91,  89,  31,  82, 154,  28,  84,  87,  30,
        78, 155,  74,  58,  44,  48,  43,  42,  52,  40,  57,  38,  73,
        62,  64,  66,  26,  36,  69,  70,  93, 100,  25, 117, 121, 124,
       126, 128, 130,   6, 133, 134, 137, 138,   5,   3, 145, 146, 147,
       149, 153, 119,  11, 116, 106,  22,  99, 101, 102,  19,  46,  12,
       115,  37,   4,  23,  35,  24,  45,  41,  33,  21,   8,  20,   9,
        34,  39,  16,  15,  13,  17,  77,  47, 103, 105, 107, 109, 110,
       111, 112, 113, 114, 120, 123, 125, 127, 131, 132, 141, 142, 143,
       148, 150, 104,  98,  49,  97,  50,  51,  56,  60,  61,  65,  68,
        71,  75,  76,  80,  81,  85,  86,  88,  90,  94,  95,  96,   0])

In [115]:
# 높은 순으로 정렬된(내림차순 정렬된) 비교 행 위치 인덱스 값 구해서
# 첫 번째(0번 레코드)의 비교 행 위치 인덱스 값만 샘플로 추출
tags_sim_sorted_ind = tags_sim.argsort()[:, ::-1]
print(tags_sim_sorted_ind[:1]) # [0]


# 결과
# 0번 레코드의 경우 자신인 0번 레코들 제외하면
# 142번 레코드가 가장 유사도가 높고
# 그 다음이 110 레코드이고, ...
# 가장 유사도가 낮은 레코드는 155 레코드라는 의미

# 이제  tags_sim_sorted_ind 객체는
# 각 레코드의 장르 코사인 유사도가 가장 높은 순으로 정렬된
# 타 레코드의 위치 인덱스 값을 가지고 있음
# 이 위치 인덱스를 이용해서 언제든지 특정 레코드와 
# 코사인 유도가 높은 다른 레코드 추출 가능

[[  0 142 110 107  98 127 131 132  86 114  76  61 141  80  41  39  35  34
   17 148  24  21  95 111  13 104 150  90  45  88  85  81  71 112  96  97
   33 109  77  46  47  36  15  89 145 146 147  19  84  64  22  30  65  66
   25  26 137  32  74  73  31  92 138  38 117   4   5  42 128 123 102 101
  100  99  56 125  57 120  44 140  40  43 143 144  37 116  29  28   1   2
    3   6   7   8   9  10  11  12 153  14  16 152  18  20  23 149  27 151
   58  48 126  83 134 133  87 130 129  91  93  94 124 135 103 105 106 122
  108 121 119 118 113  82 136  49 139  50  51  52  53  54  55 115  59  60
   62  79  63  67  68  69  70  72  75 154  78 155]]


### 특정 챔피언과 태그가 유사한  챔피언을 반환하는 함수 생성

In [116]:
# def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
#     # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
#     title_movie = df[df['title'] == title_name]
    
#     # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
#     # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
#     title_index = title_movie.index.values
#     similar_indexes = sorted_ind[title_index, :(top_n)]
    
#     # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
#     #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
#     print(similar_indexes)
#     similar_indexes = similar_indexes.reshape(-1)
    
#     return df.iloc[similar_indexes]

In [119]:
# 태그 유사도에 따라 챔피언을 추천하는 함수

def find_sim_id(lol_df, sorted_ind, title_name, top_n=10):
    # titile 데이터 추출
    title_id = lol_df[lol_df['id'] == title_name]
    
    # title_name에 해당되는 데이터프레임 index 객체를 배열로 반환하고
    # 유사도 순으로 top_n개의 index 추출
    title_index = title_id.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력 
    # 2차원 데이터를 1차원 array로 변환
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    # index에 해당되는 데이터프레임 반환
    return lol_df.loc[similar_indexes]

In [121]:
# 태그 'Assassin'와 챔피언별로 유사한 챔피언 10개 추천
similar_id = find_sim_id(df, tags_sim_sorted_ind, 'Aatrox', 10)
similar_id[['id', 'tags']]

[[  0 142 110 107  98 127 131 132  86 114]]


,id,tags
0,Aatrox,"[Fighter, Tank]"
142,Warwick,"[Fighter, Tank]"
110,Shyvana,"[Fighter, Tank]"
107,Sett,"[Fighter, Tank]"
98,Renekton,"[Fighter, Tank]"
127,Trundle,"[Fighter, Tank]"
131,Udyr,"[Fighter, Tank]"
132,Urgot,"[Fighter, Tank]"
86,Olaf,"[Fighter, Tank]"
114,Skarner,"[Fighter, Tank]"


In [39]:
df

,id,tags,new_tags
0,Aatrox,"['Fighter', 'Tank']","['Fighter', 'Tank']"
1,Ahri,"['Mage', 'Assassin']","['Mage', 'Assassin']"
2,Akali,['Assassin'],['Assassin']
3,Akshan,"['Marksman', 'Assassin']","['Marksman', 'Assassin']"
4,Alistar,"['Tank', 'Support']","['Tank', 'Support']"
...,...,...,...
151,Zed,['Assassin'],['Assassin']
152,Ziggs,['Mage'],['Mage']
153,Zilean,"['Support', 'Mage']","['Support', 'Mage']"
154,Zoe,"['Mage', 'Support']","['Mage', 'Support']"


--> 함수가 오류난거같음

In [ ]:
df[['name', 'title', 'attack']].sort_values('attack', ascending=False)[:10]

,name,title,attack
143,Xayah,the Rebel,10
48,Jhin,the Virtuoso,10
30,Fiora,the Grand Duelist,10
74,Master Yi,the Wuju Bladesman,10
57,Kayn,the Shadow Reaper,10
134,Vayne,the Night Hunter,10
128,Tryndamere,the Barbarian King,10
86,Olaf,the Berserker,9
23,Draven,the Glorious Executioner,9
122,Talon,the Blade's Shadow,9


- 컬럼설명 부분이 명확하지 않다
- 태그가 유사한 챔피언들 함수 생성 함수가 오류난거 같음
- 수업시간에 배웠던 가중치를 어디에 줘야할지 모르겠다